In [1]:
# This script initially tested out Alina's model and rewrote iPAU.xml as her model. Uncomment the last block to do so again, 
# otherwise, this script is now just for testing the reactions added by Alina to see their impact on biomass.
# Initialization:
import cobra
import cobra.test
from cobra.flux_analysis import (single_gene_deletion)
import os
from os.path import join
import pandas
from time import time
import csv

# Load LJD functions which includes changeMedia_PA_LJD(model, media)
from LJD_Functions import *

# Had to install libsbml to load an xml file
# pip install python-libsbml in Python/Scripts folder
# website: http://sbml.org/Software/libSBML/Downloading_libSBML#Using_pip_from_PyPI
import libsbml

# Other packages
from copy import *
import scipy


In [2]:
# Load the model from Alina
model_start = cobra.io.read_sbml_model('iPAU_renz_updated.xml')

In [3]:
# Check change media function 
# They appear to work correctly. Values are slightly off iPau1129 probably due to reactions Alina added

# No media
model_noMedia = changeMedia_PA_LJD(model_start, 0)
sol_noMedia = model_noMedia.optimize()
print('no media', sol_noMedia.objective_value)

# LB Media
model_LB = changeMedia_PA_LJD(model_start, 1)
sol_LB = model_LB.optimize()
print('LB', sol_LB.objective_value)

# SCFM
model_SCFM = changeMedia_PA_LJD(model_start, 2)
sol_SCFM = model_SCFM.optimize()
print('SCFM', sol_SCFM.objective_value)

# Minimal + Succinate
model_MM_succ = changeMedia_PA_LJD(model_start, 3, ['EX_cpd00036_e'])
sol_MM_succ = model_MM_succ.optimize()
print('minimal with succinate', sol_MM_succ.objective_value)



There is an `e` compartment but it does not look like it is the actual external compartment.


all exchange bounds set to [0,1000]
no media 1.6315569623170383e-16


There is an `e` compartment but it does not look like it is the actual external compartment.


LB 16.51651595130235


There is an `e` compartment but it does not look like it is the actual external compartment.


SCFM 18.958087874537757


There is an `e` compartment but it does not look like it is the actual external compartment.


minimal with succinate 0.8302451031411117


In [5]:
# Quick changes 12/13/18

# Make sure all exchanges have bounds [0,1000]
model_end = changeMedia_PA_LJD(model_start, 0)

# Turn off all reactions added by Alina temporarily by changing bounds to 0

rxns_AR = ['rxn05964',
'rxn06079',
'rxn06087',
'rxn05901',
'rxn06140',
'rxn06043',
'rxn04684',
'rxn00538',
'rxn05829',
'rxn05824',
'rxn00182',
'rxn02749',
'rxn02751',
'rxn00735',
'rxn03062',
'rxn01208',
'rxn03436',
'rxn03435',
'rxn02187',
'rxn03068',
'rxn03437',
'rxn00804',
'rxn01573',
'rxn00904']

# Eventually reset these
rxn_AR_bounds = [[0,1000],
[0,1000],
[0,1000],
[0,1000],
[0,1000],
[0,1000],
[0,1000],
[0,1000],
[0,1000],
[0,1000],
[-1000,1000],
[0,1000],
[0,1000],
[0,1000],
[-1000,1000],
[0,1000],
[-1000,1000],
[-1000,1000],
[-1000,1000],
[-1000,1000],
[0,1000],
[-1000,1000],
[0,1000],
[-1000,1000]]

for r in model_end.reactions:
    if r.id in rxns_AR:
        r.bounds = [0,0]


There is an `e` compartment but it does not look like it is the actual external compartment.


all exchange bounds set to [0,1000]


In [6]:
# Turning the bounds off of Alina's reactions restores the growth to iPAU1129 levels across media conditions

# No media
model_noMedia = changeMedia_PA_LJD(model_end, 0)
sol_noMedia = model_noMedia.optimize()
print('no media', sol_noMedia.objective_value)

# LB Media
model_LB = changeMedia_PA_LJD(model_end, 1)
sol_LB = model_LB.optimize()
print('LB', sol_LB.objective_value)

# SCFM
model_SCFM = changeMedia_PA_LJD(model_end, 2)
sol_SCFM = model_SCFM.optimize()
print('SCFM', sol_SCFM.objective_value)

# Minimal + Succinate
model_MM_succ = changeMedia_PA_LJD(model_end, 3, ['EX_cpd00036_e'])
sol_MM_succ = model_MM_succ.optimize()
print('minimal with succinate', sol_MM_succ.objective_value)



There is an `e` compartment but it does not look like it is the actual external compartment.


all exchange bounds set to [0,1000]
no media 1.823953446194033e-16


There is an `e` compartment but it does not look like it is the actual external compartment.


LB 15.729762115657628


There is an `e` compartment but it does not look like it is the actual external compartment.


SCFM 15.508373240531986


There is an `e` compartment but it does not look like it is the actual external compartment.


minimal with succinate 0.7802886729466981


In [25]:
# Turn exchange reactions back on for Alina's model
for rxns in rxns_AR:
    model_end.reactions.get_by_id(rxns).bounds = rxn_AR_bounds[rxns_AR.index(rxns)]
    print('turned back on',rxns, model_end.reactions.get_by_id(rxns).bounds)
    print('before turning off',rxns_AR[rxns_AR.index(rxns)], rxn_AR_bounds[rxns_AR.index(rxns)])

turned back on rxn05964 (0, 1000)
before turning off rxn05964 [0, 1000]
turned back on rxn06079 (0, 1000)
before turning off rxn06079 [0, 1000]
turned back on rxn06087 (0, 1000)
before turning off rxn06087 [0, 1000]
turned back on rxn05901 (0, 1000)
before turning off rxn05901 [0, 1000]
turned back on rxn06140 (0, 1000)
before turning off rxn06140 [0, 1000]
turned back on rxn06043 (0, 1000)
before turning off rxn06043 [0, 1000]
turned back on rxn04684 (0, 1000)
before turning off rxn04684 [0, 1000]
turned back on rxn00538 (0, 1000)
before turning off rxn00538 [0, 1000]
turned back on rxn05829 (0, 1000)
before turning off rxn05829 [0, 1000]
turned back on rxn05824 (0, 1000)
before turning off rxn05824 [0, 1000]
turned back on rxn00182 (-1000, 1000)
before turning off rxn00182 [-1000, 1000]
turned back on rxn02749 (0, 1000)
before turning off rxn02749 [0, 1000]
turned back on rxn02751 (0, 1000)
before turning off rxn02751 [0, 1000]
turned back on rxn00735 (0, 1000)
before turning off rxn

In [26]:
# Export model
#cobra.io.write_sbml_model(model_end, "iPAU.xml")
#cobra.io.save_yaml_model(model_end, "iPAU.yml")

